# L'héritage en python #

Repartons de notre classe User :

In [ ]:
from hashlib import blake2b
import os

class User:
    """ Example de champs 
    __init__ est le constructeur, id, name, __salt, __password des champs """
    def __init__(self, id, name, password) -> None :
        self.id = id
        self.name = name
        self.__salt = os.urandom(blake2b.SALT_SIZE)
        self.__password = self.__crypt_pwd(password)

    def __crypt_pwd(self, password):
        h1 = blake2b(salt=self.__salt)
        return h1.update(password.encode(encoding = 'UTF-8'))
    
    def check_pwd(self, password):
        return self.__password == self.__crypt_pwd(password)

Définissons une classe Admin qui héritera de User :

In [ ]:
class Admin(User):
    def manage(self):
        print('I am a Super administrator!')

In [ ]:
root = Admin(1, 'root', 'toor')
root.check_pwd('toor')

In [ ]:
root.manage()

Nous pouvons avoir deux classes différentes héritant d’une même mère

In [ ]:
class Guest(User):
    def __init__(self, id, name):
        super().__init__(id, name, '')

    def check_pwd(self, password):
        return True

In [ ]:
guest = Guest(3, 'Guest')
guest.check_pwd('password')

En Python, la fonction isinstance permet de tester si un objet est l’instance d’une certaine classe.

In [ ]:
isinstance(root, Admin)

In [ ]:
isinstance(root, User)

Reprenons notre pseudo-forum :

In [ ]:
import os
import datetime
from hashlib import blake2b

class User:
    """ Example de champs 
    __init__ est le constructeur, id, name, __salt, __password des champs """
    def __init__(self, id, name, password) -> None :
        self.id = id
        self.name = name
        self.__salt = os.urandom(blake2b.SALT_SIZE)
        self.__password = self.__crypt_pwd(password)

    def __crypt_pwd(self, password):
        h1 = blake2b(salt=self.__salt)
        return h1.update(password.encode(encoding = 'UTF-8'))
    
    def check_pwd(self, password):
        return self.__password == self.__crypt_pwd(password)

    def new_thread(self, title, message):
        return Thread(title, self, message)

    def answer_thread(self, thread, message):
        thread.answer(self, message)

class Post:
    def __init__(self, author, message):
        self.author = author
        self.message = message
        self.date = datetime.datetime.now()

    def format(self):
        date = self.date.strftime('le %d/%m/%Y à %H:%M:%S')
        return '<div><span>Par {} {}</span><p>{}</p></div>'.format(self.author.name, date, self.message)

class Thread(Post):
    def __init__(self, title, author, message):
        super().__init__(author, message)
        self.title = title
        self.posts = []

    def answer(self, author, message):
        self.posts.append(Post(author, message))

    def format(self):
        posts = ['<div>' + self.title + '</div']
        posts += [super().format()]
        posts += [p.format() for p in self.posts]
        return '\n'.join(posts)

if __name__ == '__main__':
    john = User(1, 'john', '12345')
    peter = User(2, 'peter', 'toto')
    thread = john.new_thread('Bienvenue', 'Bienvenue à tous')
    peter.answer_thread(thread, 'Merci')
    pol = User(3,'pol','123')
    pol.answer_thread(thread,'Merci John et Peter')
    print(thread.format())